# **Token & Embedding 자세히 살펴보기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



# **Tokenization**

In [ ]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

## LLM 토큰화 확인


**1. 모델 로드하고 토크나이저 지정하기**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

**2.입력 프롬프트를 토큰으로 나누기**

In [ ]:
prompt = '''Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|>'''

# 입력 프롬프트를 토큰으로 나눕니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20   # 토큰을 20개 생성
)

# 출력을 프린트합니다.
print(tokenizer.decode(generation_output[0]))  # decode 메서드: 토큰ID를 실제 텍스트로 변환

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah


In [ ]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001]], device='cuda:0')


In [ ]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.


Exp
lain
how
it
happened
.
<|assistant|>


In [ ]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,
           406,  6225, 11763,   363,   278, 19906,   292,   341,   728,   481,
            13,    13, 29928,   799, 19235]], device='cuda:0')

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


## **토크나이저가 텍스트를 분할하는 방법**

1. 모델 설계시 모델 작성자가 토큰화 방법을 선택
    - GTP 모델 : BPE(byte pair encoding)
    - BERT 모델 : WordPiece
2. 토큰화 방법을 선택한 후에 어휘사전 크기와 특수 토큰 같은 토크나이저 설계상의 여러 가지 선택을 해야함
3. 토크나이저는 특정 데이터셋에서 훈련하여 해당 데이터셋을 표현하는 최상의 어휘사전을 구축해야 함

## **토큰 종류**

- **단어 토큰**
    - word2vec와 같은 초기 토큰화에 사용됨 --> 현재는 덜 사용됨
    - 지금도 추천 시스템과 같은 곳에서 사용
    - (단점: 훈련된 후에 데이터셋에 새롭게 추가된 단어는 처리할 수 없다.-->되도록 많은 어휘사전을 만들어야 한다.)

- **부분단어 토큰** (완전단어+부분완전단어 포함)
    - 새로운 단어를 (어휘사전에 포함되어 있을 가능성이 높은) 더 작은 단위로 나눈다.
    - **평균적으로 토큰당 세 개의 문자로 구성됨**
- **문자 토큰**
    - 대체할 원시 문자가 있기 때문에 새로운 단어를 잘 처리할 수 있다.
    - 토큰화는 쉽지만 모델링은 어렵다.--> 문자를 조합하는 정보까지 모델링해야함)
- **바이트 토큰**
    - 유니코드 문자를 표현하는 바이트로 토큰을 분할하는 방법
    - '토큰화-프리 인코딩' 라고 부름
    - 다국어 환경에서 경쟁력이 있다고 봄

## **훈련된 LLM 토큰나이저 비교하기**


- **중점 비교 요소**
    - **토큰화 방법** :  BEP, SentencePiece, WordPiece
    - **토크나이저 파라미터**
        - 어휘사전 크기 : 토크나이저가 어휘사전에 얼마나 많은 토큰을 포함할 건가?
        - 특수 토큰 : 모델이 추적해야할 특수 토큰은 무엇인가?
        - 대소문자 : 영어와 같은 대소문자를 어떻게 다뤄야 할까?

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        # 텍스트를 인코딩한 후 다시 디코딩했을 때 원본 텍스트와 동일해지려면
        # clean_up_tokenization_spaces를 False로 지정해야 합니다.
        # 현재 이 매개변수의 기본값은 None(True에 해당)이며
        # transformers 4.45에서 True로 바뀔 예정입니다.
        # https://github.com/huggingface/transformers/issues/31884
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )



---



## **한국어 특화된 토크나이저**

- **한글의 특수성**:
    - **교착어**: 조사가 붙어 단어 형태가 무한대로 변형 ("학교", "학교가", "학교에서", "학교로부터"...)
    - **띄어쓰기 불규칙**: SNS, 댓글 등에서 띄어쓰기가 일관되지 않음
    - **자모 조합**: 초성+중성+종성이 결합되어 하나의 음절 형성 (ex: ㄱ+ㅏ+ㅁ = 감)

- 발전 과정:
    - 2013-2016: KoNLPy, 형태소 분석기 (Mecab, Okt) 등장
    - 2018: 다국어 BERT에 한국어 포함되었으나 성능 부족
    - 2020: KoBERT (SKT) 출시 - 한국어 Wikipedia로 학습
    - 2021: KoGPT, KoBART 등 한국어 특화 모델 등장
    - 2022-2024: KoAlpaca, Polyglot-Ko, KULLM 등 다양한 한국어 LLM 개발

- 주요 참고자료:
    - SKT KoBERT: https://github.com/SKTBrain/KoBERT
    - Kakao KoGPT: https://github.com/kakaobrain/kogpt

### **[미션]** 한국어 LLM 찾아 토큰화 해보기.
- 허깅페이스에서 사전학습된 한국어 LLM을 여러 개(2개 이상) 찾고 위와 같이 토큰화 & 비교해보세요.

In [ ]:
ko_text = """
    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,
"""

# 예제 : SKT의 KoBERT - 비교적 안정적
show_tokens(ko_text, "skt/kobert-base-v1")

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

 ᄋ [UNK] ᄒ [UNK] ᄋ [UNK] !  [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᅵ [UNK] ᄋ [UNK]  ᄀ [UNK] ᄒ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᅵ [UNK] . ,  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ [UNK] ᄋ ᅵ  [UNK] ᄒ [UNK] ᅵ [UNK]  [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] .  [UNK] ,  ᄏ [UNK] ᄏ [UNK] ᄋ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK]  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK] IT  ᄀ ᅵ ᄋ [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] . ,  [UNK] ᄋ [UNK] ᄋ [UNK] ᅵ ( N L P ) [UNK]  ᄏ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᄋ [UNK]  ᄋ [UNK] ᄋ [UNK]  ᄋ ᅵ ᄒ [UNK] ᄒ [UNK]  ᄒ [UNK]  ᄀ ᅵ [UNK] ᄋ ᅵ [UNK] . ,  [UNK] [UNK] 

In [ ]:
# EleutherAI의 Polyglot-Ko 시리즈
show_tokens(ko_text, "EleutherAI/polyglot-ko-1.3b")

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]


        안녕 하 세요 !  저 는  인공지능 을  공부 하고  있 습니다 . , 
        한글 은  세종 대왕 이  창 제 하 신  문자 입니다 .   � � � � , 
        카카오 와  네이버 는  한국 의  대표 적 인  IT  기업 입니다 . , 
        자연 어 처리 ( NL P ) 는  컴퓨터 가  인간 의  언어 를  이해 하 도록  하 는  기술 이 다 . , 
 

In [ ]:
# Beomi의 KcBERT - 댓글 데이터 특화
show_tokens(ko_text, "beomi/OPEN-SOLAR-KO-10.7B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

<s>  
     안 녕 하세요 ! 저는 인공지능 을 공부 하고 있습니다 ., 
     한 글 은 세종 대 왕 이 창 제 하신 문자 입니다 .  😊 , 
     카 카오 와 네이버 는 한국의 대표적인 IT 기업 입니다 ., 
     자연 어 처리 ( N LP ) 는 컴퓨터 가 인간의 언 어를 이해 하도록 하는 기술 이다 ., 
 

In [ ]:
# monologg의 KoBERT (Transformers 호환)

show_tokens(ko_text, "monologg/kobert")


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert .
 You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

[CLS] 안 녕 하세요 ! 저 는 인 공 지 능 을 공부 하고 있습니다 . , 한 글 은 세종 대 왕 이 창 제 하 신 문자 입니다 .  [UNK] , 카카오 와 네이버 는 한국의 대표적인 IT 기업 입니다 . , 자연 어 처리 ( N L P ) 는 컴퓨터 가 인간 의 언 어 를 이해 하도록 하는 기술 이다 . , [SEP] 

In [ ]:
# https://huggingface.co/ruatnec/upstageai-6team-kobge-model

show_tokens(ko_text, 'ruatnec/upstageai-6team-kobge-model')

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

<s> 안녕하세요 ! 저는 인공지능 을 공부 하고 있습니다 . , 한글 은 세종 대 왕 이 창 제 하신 문자 입니다 .  😊 , 카 카 오 와 네이버 는 한국의 대표적인 IT 기업 입니다 . , 자연 어 처리 ( NL P ) 는 컴퓨터 가  인간의 언어 를 이해 하도록 하는 기술 이다 . , </s> 

## 10/01_미션 정리

In [ ]:
from transformers import AutoTokenizer

# 비교할 모델 6개
model_names = [
    "skt/kobert-base-v1",
    'EleutherAI/polyglot-ko-1.3b',
    "klue/bert-base",
    "kykim/bert-kor-base",
    'ruatnec/upstageai-6team-kobge-model',
    'monologg/kobert'
]

sentence = "오늘 점심은 오리고기 김치볶음밥을 먹었다."

for i, model_name in enumerate(model_names, start=1):
    print('\n')
    print(f"\n✅ {i}. {model_name} ###")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = tokenizer.tokenize(sentence)
    print(tokens)
    show_tokens(sentence, model_name)





✅ 1. skt/kobert-base-v1 ###
['▁', 'ᄋ', 'ᅩ늘', '▁', '점ᄉ', 'ᅵ', 'ᆷ', 'ᄋ', 'ᅳᆫ', '▁', 'ᄋ', 'ᅩᄅ', 'ᅵ', 'ᄀ', 'ᅩ', 'ᄀ', 'ᅵ', '▁', 'ᄀ', 'ᅵ', 'ᆷᄎ', 'ᅵ', '볶', 'ᄋ', 'ᅳᆷ밥', 'ᄋ', 'ᅳᆯ', '▁', '먹', 'ᄋ', 'ᅥᆻ다', '.']
 ᄋ [UNK]  [UNK] ᅵ [UNK] ᄋ [UNK]  ᄋ [UNK] ᅵ ᄀ [UNK] ᄀ ᅵ  ᄀ ᅵ [UNK] ᅵ [UNK] ᄋ [UNK] ᄋ [UNK]  [UNK] ᄋ [UNK] . [UNK] [UNK] 


✅ 2. EleutherAI/polyglot-ko-1.3b ###
['ìĺ¤ëĬĺ', 'ĠìłĲìĭ¬', 'ìĿĢ', 'Ġìĺ¤ë¦¬', 'ê³łê¸°', 'Ġê¹Ģì¹ĺ', 'ë³¶ìĿĮ', 'ë°¥', 'ìĿĦ', 'Ġë¨¹', 'ìĹĪ', 'ëĭ¤', '.']
오늘  점심 은  오리 고기  김치 볶음 밥 을  먹 었 다 . 


✅ 3. klue/bert-base ###
['오늘', '점심', '##은', '오리', '##고기', '김치', '##볶음', '##밥', '##을', '먹', '##었', '##다', '.']
[CLS] 오늘 점심 ##은 오리 ##고기 김치 ##볶음 ##밥 ##을 먹 ##었 ##다 . [SEP] 


✅ 4. kykim/bert-kor-base ###
['오늘', '점심', '##은', '오리고', '##기', '김치', '##볶음밥', '##을', '먹었다', '.']
[CLS] 오늘 점심 ##은 오리고 ##기 김치 ##볶음밥 ##을 먹었다 . [SEP] 


✅ 5. ruatnec/upstageai-6team-kobge-model ###
['▁오늘', '▁점심', '은', '▁오', '리고', '기', '▁김', '치', '볶', '음', '밥', '을', '▁먹', '었다', '.']
<s> 오늘 점심 은 오 리고 기 김 치 볶 음 



---

